# Wiki whā (4) tutorial Python notebook

Use this notebook to help you code and calculate answers to the Week 4 tutorial questions. We'll assume you're familar with Python operations now, so get straight to it.

Remeber, useful equations for the tutorial should be in your class notes or the lecture summary notes on Learn.

Ready, set, code!


In [1]:
# Load packages - add any others you might need
import pandas as pd
import numpy as np
import math

In [2]:
# Begin code! 
# Question 1
cap = 30 #MW
T_h = 171+273
T_c = 15+273

#calculate carnot and PP efficiency
eta_c = 1-(T_c/T_h)
eta_PP = 0.5*eta_c

print('The carnot efficiency is {eff_c:.0%} and the power plant efficiency is {eff:.0%}'.format(eff_c=eta_c,eff=eta_PP))

#power output at 35C and decrease in efficiency
T_c2 = 35+273
eta_PP2 = 0.5*(1-(T_c2/T_h))
eff_decrease = 1-(eta_PP2/eta_PP)

print('At 35 degrees C, the decrease in efficiency is {eff_d:.0%}'.format(eff_d=eff_decrease))

#percent increase in efficiency with spray cooling
T_wb = 13+273
T_c3 = T_c2 - (T_c2-T_wb)*0.6
eta_PP3 = 0.5*(1-(T_c3/T_h))
eff_increase = (eta_PP3/eta_PP2)-1

print('With spray cooling, the efficiency increases {eff_i:.0%}'.format(eff_i=eff_increase))

#turbine efficiency
cap_t = cap/5 * 1000 #kW = kJ/s
H_steam = 2804 #kJ/kg
x_iso = 0.74
x_act = 0.82
H_v = 2591 #kJ/kg
H_l = 209 #kJ/kg

H_iso = x_iso*H_v + (1-x_iso)*H_l
H_act = x_act*H_v + (1-x_act)*H_l

eta_turbine = (H_steam - H_act)/(H_steam - H_iso)

print('The efficiency of the turbine is {eff_t:.0%}'.format(eff_t=eta_turbine))

#calculate mass flow rate
m_dot = cap_t/(H_steam-H_act) #kg/s

print('The mass flow rate through each turbine is {m:.2f} kg/s'.format(m=m_dot))

The carnot efficiency is 35% and the power plant efficiency is 18%
At 35 degrees C, the decrease in efficiency is 13%
With spray cooling, the efficiency increases 10%
The efficiency of the turbine is 77%
The mass flow rate through each turbine is 9.35 kg/s


In [3]:
# Question 2
T_g = 12+273
T_house = 21+273
T_h = 34.8+273
T_c = -34.4+273

#In summer
COP_g = T_g/(T_house-T_g)
COP_a = T_house/(T_h-T_house)

print('In summer, the GSHP has a COP of {cop_g:.2f} and the air HP has a COP of {cop_a:.2f}'.format(cop_g=COP_g, cop_a=COP_a))

#In winter
COP_g = T_house/(T_house-T_g)
COP_a = T_house/(T_house-T_c)

print('In winter, the GSHP has a COP of {cop_g:.2f} and the air HP has a COP of {cop_a:.2f}'.format(cop_g=COP_g, cop_a=COP_a))

In summer, the GSHP has a COP of 31.67 and the air HP has a COP of 21.30
In winter, the GSHP has a COP of 32.67 and the air HP has a COP of 5.31


In [4]:
# Question 2 continued
heatloss = 105 #MJ/h

# load data
temp_df = pd.read_csv('calgary_hourly.csv')

#calculate COPs at each hour
temp_df['COP_a'] = [T_house/(T_house-(t+273)) * 0.7 if t < 21 else (t/((t+273)-T_house) * 0.7 if t > 21 else 0) for t in temp_df['temperature_degC']]
temp_df['COP_g'] = [T_house/(T_house-T_g) * 0.7 if t < 21 else T_g/(T_house-T_g)*0.7 for t in temp_df['temperature_degC']]

#calculate electricity 
temp_df['e_air'] = [heatloss/60/60/cop*1000 if cop>0 else 0 for cop in temp_df['COP_a']]
temp_df['e_ground'] = [heatloss/60/60/cop*1000 for cop in temp_df['COP_g']]

#total costs
cost_air = temp_df['e_air'].sum()*0.14 #$
cost_ground = temp_df['e_ground'].sum()*0.14 #$

print('The air HP would cost ${c_a:.2f} per year and the GSHP would cost ${c_g:.2f} per year'.format(c_a=cost_air,c_g=cost_ground))



The air HP would cost $3546.31 per year and the GSHP would cost $1569.13 per year


In [5]:
#CHALLENGE
temp_df['date'] = pd.to_datetime(temp_df['date'])
temp_df['time_local'] = pd.to_datetime(temp_df['time_local'])

temp_df['month'] = [m.month for m in temp_df['date']]
temp_df['hour'] = [h.hour for h in temp_df['time_local']]

#set up cost structure
cost_w = [0.06,0.06,0.06,0.06,0.06,0.06,0.16,0.16,0.16,0.16,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.2,0.2,0.2,0.2,0.2,0.2,0.06]
cost_s = [0.04,0.04,0.04,0.04,0.04,0.04,0.14,0.14,0.14,0.14,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.18,0.18,0.18,0.18,0.18,0.18,0.04]

#summarise our electricity data for air and ground heating, in kWh
elec_air = pd.pivot_table(temp_df,index=['month'],columns=['hour'],values=['e_air'],aggfunc='sum')
elec_ground = pd.pivot_table(temp_df,index=['month'],columns=['hour'],values=['e_ground'],aggfunc='sum')

cost_a = 0
cost_g = 0

for c in range(0,len(elec_air.index)):
    if c in [0,1,2,9,10,11]:
        cost_a += np.sum(elec_air.iloc[c,:]*cost_w)
        cost_g += np.sum(elec_ground.iloc[c,:]*cost_w)
    else:
        cost_a += np.sum(elec_air.iloc[c,:]*cost_s)
        cost_g += np.sum(elec_ground.iloc[c,:]*cost_s)

#some rounding differences between excel and python pivoting -- acceptable
print('The air HP would cost ${c_a:.2f} per year and the GSHP would cost ${c_g:.2f} per year'.format(c_a=cost_a,c_g=cost_g))


The air HP would cost $2868.10 per year and the GSHP would cost $1221.31 per year
